In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import openai
import pandas as pd

In [ ]:
%cd /content/drive/MyDrive/자기주도연구

/content/drive/MyDrive/자기주도연구


In [ ]:
!ls

 all_negative_reason.csv		  comments_중증외상센터_keyword.csv  '댓글 정리.ipynb'
'comments_나의 완벽한 비서_keyword.csv'   filtering.ipynb		     '데이터 수집'
 comments_모텔캘리포니아_keyword.csv	  story_negative_trends_summary.md   '데이터 통계.ipynb'
 comments_오징어게임2_keyword.csv	 '관련없는 댓글 삭제(전처리)'	     '세부적 이유 추출'
 comments_옥씨부인전_keyword.csv	 '긍정부정 이유(keyword).ipynb'      '언급횟수 그래프'
 comments_원경_keyword.csv		 '댓글데이터 수집.ipynb'


In [ ]:
from openai import OpenAI
import json

In [ ]:
com1 = pd.read_csv('세부적 이유 추출/스토리/나의 완벽한 비서/positive/나의 완벽한 비서_story_positive_reason.csv')
com2 = pd.read_csv('세부적 이유 추출/스토리/모텔 캘리포니아/positive/모텔캘리포니아_story_positive_reason.csv')
com3 = pd.read_csv('세부적 이유 추출/스토리/오징어게임2/positive/오징어게임2_story_positive_reason.csv')
com4 = pd.read_csv('세부적 이유 추출/스토리/옥씨부인전/positive/옥씨부인전_story_positive_reason.csv')
com5 = pd.read_csv('세부적 이유 추출/스토리/원경/positive/원경_story_positive_reason.csv')
com6 = pd.read_csv('세부적 이유 추출/스토리/중증외상센터/positive/중증외상센터_story_positive_reason.csv')

In [ ]:
comment_list = pd.concat([com1, com2, com3, com4, com5, com6], ignore_index=True)

In [ ]:
# 유사 or 동일한 카테고리는 하나로 묶기
def clear_actor(col1, col2):
  global comment_list
  plus_comment = comment_list.loc[col1,"reason"]+comment_list.loc[col2,"reason"]
  comment_list.at[col1,"reason"] = plus_comment
  plus_count = comment_list.loc[col1,"count"]+comment_list.loc[col2,"count"]
  comment_list.at[col1,"count"] = plus_count

## 연기 세부 요소별

In [ ]:
comment_list

,story_element,reason,count
0,감정적 반응,"['달달한 로맨스가 있다', '드라마를 보면서 정말 설레였다', '킬포 제대로 잡았...",94
1,서사 몰입도,"['이해가 가는 드라마다', '영상 막판 보기 전에 쓴 글인데 끝까지 보고 놀랐다....",171
2,전개 방식,"['전개가 빨라서 좋다', '전개가 빠르다', '스토리 전개 속도가 괜찮음', '잔...",62
3,주제,"['부당함에 대한 이야기가 담겨있다', '딸을 위해 커리어를 포기할 수 있는 아빠와...",41
4,플롯 구조,"['줄거리만 대충대충 지나가는대도 상당히 재미있거덩', '역클리셰 맛집이다', '둘...",74


In [ ]:
ae=[9,14,19,23,28]

In [ ]:
print(comment_list.loc[ae,"story_element"])

9     플롯 구조
14    플롯 구조
19    플롯 구조
23    플롯 구조
28    플롯 구조
Name: story_element, dtype: object


In [ ]:
for i in ae:
  clear_actor(4,i)
  comment_list = comment_list.drop(index=i)
  comment_list

In [ ]:
comment_list.to_csv("all_positive_reason.csv", index=False)

In [ ]:
comment_list = pd.read_csv("all_positive_reason.csv")

In [ ]:
comment_list

,story_element,reason,count
0,감정적 반응,"['달달한 로맨스가 있다', '드라마를 보면서 정말 설레였다', '킬포 제대로 잡았...",94
1,서사 몰입도,"['이해가 가는 드라마다', '영상 막판 보기 전에 쓴 글인데 끝까지 보고 놀랐다....",171
2,전개 방식,"['전개가 빨라서 좋다', '전개가 빠르다', '스토리 전개 속도가 괜찮음', '잔...",62
3,주제,"['부당함에 대한 이야기가 담겨있다', '딸을 위해 커리어를 포기할 수 있는 아빠와...",41
4,플롯 구조,"['줄거리만 대충대충 지나가는대도 상당히 재미있거덩', '역클리셰 맛집이다', '둘...",74


In [ ]:
def generate_summary(trend_data):
    client = OpenAI(
      api_key = "my_gpt_api_key"
    )

    """GPT를 사용하여 스토리 요소별 트렌드 요약 생성"""
    prompt = f"""
    다음은 시청자들의 반응을 포함하는 데이터셋입니다. 이 데이터를 기반으로 K-드라마에서 사람들이 어떤 부분을 선호하고, 긍정적인 반응을 보이는지 트렌드를 분석하세요.

    {trend_data.to_string(index=False)}

    분석할 내용:
    1. 시청자들이 선호하는 부분
    2. 전반적인 감상 트렌드 (예: 스토리 흐름, 플롯 구조, 전개 방식 등)
    3. 앞으로 드라마 시장에서 계속 반영되었으면 하는 부분

    형식:
    - **주요 트렌드 요약**: (트렌드 내용)
    - **시청자들이 선호하는 요소**: (예시)
    - **기타 주목할 만한 사항**: (예시)
    """

    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": "너는 드라마 스토리 분석 전문가야."},
            {"role": "user", "content": prompt}
        ],
        temperature=0.7
    )

    return response.choices[0].message.content.strip()


In [ ]:
trend_summary = generate_summary(comment_list)

In [ ]:
with open("story_positive_trends_summary.md", "w", encoding="utf-8") as f: #appned : w->a
    f.write("드라마 최신 트랜드 분석(스토리 - positive)")  # 기존 내용과 구분
    f.write(trend_summary)

print("트렌드 요약이 story_positive_trends_summary.md 파일에 추가되었습니다.")

트렌드 요약이 story_positive_trends_summary.md 파일에 추가되었습니다.
